In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [2]:
def ex_tag(sid, page):
    ### 뉴스 분야(sid)와 페이지(page)를 입력하면 그에 대한 링크들을 리스트로 추출하는 함수 ###
    
    ## 1.
    url = f"https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={sid}"\
    "#&date=%2000:00:00&page={page}"
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    a_tag = soup.find_all("a")
    
    ## 2.
    tag_lst = []
    for a in a_tag:
        if "href" in a.attrs:  # href가 있는것만 고르는 것
            if (f"sid={sid}" in a["href"]) and ("article" in a["href"]):
                tag_lst.append(a["href"])
                
    return tag_lst

In [3]:
def re_tag(sid):
    ### 특정 분야의 100페이지까지의 뉴스의 링크를 수집하여 중복 제거한 리스트로 변환하는 함수 ###
    ## 여기 바꾸면 돼  예정!!!!
    re_lst = []
    for i in tqdm(range(1001,2000)):
        lst = ex_tag(sid, i+1)
        re_lst.extend(lst)

    # 중복 제거
    re_set = set(re_lst)
    re_lst = list(re_set)
    
    return re_lst

In [4]:
all_hrefs = {}
sids = [i for i in range(100,106)]  # 분야 리스트
### 분야 바꾸고 싶으면 여기 바꾸면 됨
# 각 분야별로 링크 수집해서 딕셔너리에 저장
for sid in sids:
    sid_data = re_tag(sid)
    all_hrefs[sid] = sid_data

  0%|          | 0/999 [00:00<?, ?it/s]

  0%|          | 0/999 [00:00<?, ?it/s]

  0%|          | 0/999 [00:00<?, ?it/s]

  0%|          | 0/999 [00:00<?, ?it/s]

  0%|          | 0/999 [00:00<?, ?it/s]

  0%|          | 0/999 [00:00<?, ?it/s]

In [5]:
def art_crawl(all_hrefs, sid, index):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수 
    
    Args: 
        all_hrefs(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [100: 정치, 101: 경제, 102: 사회, 103: 생활/문화, 104: 세계, 105: IT/과학]
        index(int): 링크의 인덱스
    
    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리
    
    """
    art_dic = {}
    
    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"
    
    url = all_hrefs[sid][index]
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    
    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)
    
    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)
    
    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)
    
    ## 3.
    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str
    
    return art_dic

In [6]:
# 모든 섹션의 데이터 수집 (제목, 날짜, 본문, section, url)
section_lst = [s for s in range(100, 106)]
artdic_lst = []

for section in tqdm(section_lst):
    for i in tqdm(range(len(all_hrefs[section]))):
        art_dic = art_crawl(all_hrefs, section, i)
        art_dic["section"] = section
        art_dic["url"] = all_hrefs[section][i]
        artdic_lst.append(art_dic)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/595 [00:00<?, ?it/s]

  0%|          | 0/946 [00:00<?, ?it/s]

  0%|          | 0/979 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/326 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

In [7]:
import pandas as pd

art_df = pd.DataFrame(artdic_lst)

In [ ]:
import datetime
now = datetime.datetime.now() 
art_df.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\1001_2000_{}.csv'.format(now.strftime('%Y%m%d_%H%M%S')))

NameError: name 'now' is not defined

In [9]:
art_df.sort_values(by='date', ascending=True)

,title,date,main,section,url
864,"교통안전공단, 중고차 허위매물 신고 '자동차365' 사이트 오픈",2023.06.27. 오전 10:49,신고 즉시 업체 소속된 관할 관청으로 접수\n\n\n\n한국교통안전공단 중고차 허위...,101,https://n.news.naver.com/mnews/article/421/000...
315,"‘평양의 유미’ 이젠 해외서도 못본다...구글, 北 선전 유튜브 채널 ‘폐쇄’",2023.06.27. 오전 10:50,평양의 일상을 소개하는 북한 유튜버 유미.(사진=유튜브 채널 'Olivia Nata...,100,https://n.news.naver.com/mnews/article/014/000...
2546,"폭스바겐 마이스터모터스, 28~30일 스마트테크 코리아에 'ID.4' 전시",2023.06.27. 오전 10:51,현장 상담 고객 및 출고 고객 대상다양한 경품 증정 이벤트 진행[이데일리 박민 기자...,103,https://n.news.naver.com/mnews/article/018/000...
869,"""중년에겐 자부심이자 추억""…그 남자의 애마 2대는 '포니2'",2023.06.27. 오전 10:51,현대차 울산공장 방형달 기술주임이 포니2 승용과 포니2 픽업과 함께 사진 촬영을 하...,101,https://n.news.naver.com/mnews/article/025/000...
2636,"中총리, 미국 겨냥 '디리스킹' 비판…""세계경제 회복 방해할 것""",2023.06.27. 오전 10:52,"리창, 톈진서 WTO 사무총장 회담…""개도국 합법적 권익 수호""\n\n\n\n리창 ...",104,https://n.news.naver.com/mnews/article/001/001...
...,...,...,...,...,...
2219,"이정근 측 ""사업가 진술 못 믿는다""…2심서도 혐의 부인",2023.06.28. 오후 12:27,사업가에게 10억 원의 금품을 받은 혐의 등으로 1심에서 징역 4년 6개월을 선고받...,102,https://n.news.naver.com/mnews/article/055/000...
2130,7월부터 서울지하철 10분 내 다시 타면 무료,2023.06.28. 오후 12:28,잘못 내리거나 화장실 이용 시 환승 적용1~9호선 중 서울시 관할구간 우선 도입\n...,102,https://n.news.naver.com/mnews/article/011/000...
2148,"민주노총, '尹정부 퇴진' 내달 3~15일 총파업…""위력 투쟁""(종합)",2023.06.28. 오후 12:28,"양경수 위원장, 총파업 기자 간담회…투쟁 계획 등 밝혀3일부터 산별노조 '줄파업'…...",102,https://n.news.naver.com/mnews/article/003/001...
2097,필로폰·신종 마약 판매 일당·투약자 49명 구속,2023.06.28. 오후 12:28,"울산 남부서, 총 55명 검거… 해외 마약 공급 총책 추적필로폰 3200회 투약분,...",102,https://n.news.naver.com/mnews/article/081/000...
